<a href="https://colab.research.google.com/github/YvesAugusto/Fingerprint-Verification/blob/master/transferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import cv2 as cv
import numpy as np
def loadData(f):
    trainx=[]
    testx=[]
    trainy=[]
    testy=[]

    for i in range(1, 17):
        for j in range(1,int(f*49)):
            v = np.zeros(16)
            v[i - 1] = 1
            trainx.append(cv.imread("drive/My Drive/database/" + str(i)+"_"+str(j)+".png", cv.IMREAD_GRAYSCALE))
            trainy.append(v)
        for j in range(int(f*49), 49):
            testx.append(cv.imread("drive/My Drive/database/" + str(i)+"_"+str(j)+".png", cv.IMREAD_GRAYSCALE))
            v=np.zeros(16)
            v[i-1]=1
            testy.append(v)
            
    trainx=np.array(trainx)
    testx=np.array(testx)
    trainy=np.array(trainy)
    testy=np.array(testy)
   
    return trainx,testx,trainy,testy
            
trainx,testx,trainy,testy=loadData(0.8)
print(trainx.shape, testx.shape, trainy.shape, testy.shape)
print(trainx[1])

(608, 338, 248) (160, 338, 248) (608, 16) (160, 16)
[[232 232 232 ... 232 232 232]
 [232 232 232 ... 232 232 232]
 [232 232 232 ... 232 232 232]
 ...
 [232 232 232 ... 232 232 232]
 [232 232 232 ... 232 232 232]
 [232 232 232 ... 232 232 232]]


In [4]:
train_x=[]
test_x=[]
for row in trainx:
  row=np.float32(row)
  v=cv.cvtColor(row,cv.COLOR_GRAY2RGB)
  train_x.append(v)

for row in testx:
  row=np.float32(row)
  v=cv.cvtColor(row,cv.COLOR_GRAY2RGB)
  test_x.append(v)

trainX=[]
testX=[]

for row in train_x:
  v=np.resize(row,(224,224,3))
  trainX.append(v)

for row in test_x:
  v=np.resize(row,(224,224,3))
  testX.append(v)

train_x=np.array(trainX)
test_x=np.array(testX)

print(train_x.shape)
print(test_x.shape)


(608, 224, 224, 3)
(160, 224, 224, 3)


In [0]:
from keras.applications.mobilenet import MobileNet
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, BatchNormalization, Flatten
model=MobileNet(input_shape=(224,224,3), include_top=False, weights='imagenet', classes=16, pooling=(2,2))
net= Sequential()
net.add(model)
for i in range(len(net.layers)-5):
  net.layers[i].trainable=False

net.add(Flatten())
net.add(BatchNormalization())
net.add(Dense(1536, activation='relu'))
net.add(Dense(1152, activation='relu'))
net.add(Dense(16, activation='softmax'))
sgd=optimizers.Adam(lr=1e-6*4, beta_1=0.9, beta_2=0.999, amsgrad=False)
net.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
net.fit(x=train_x, y=trainy, epochs=6, batch_size=16, validation_data=(test_x, testy))



Train on 608 samples, validate on 160 samples
Epoch 1/6
 80/608 [==>...........................] - ETA: 1:21 - loss: 3.1223 - acc: 0.0875